In [193]:
import json
import numpy as np
import pandas as pd

# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Number of notebooks

We will have 3 notebooks. First notebook will do general cleaning, second notebook will have some EDA and finally the third
notebook will have the machine learning modelling.

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record 

### Portfolio cleaning

In [194]:
portfolio = pd.read_json('resources/portfolio.json', orient='records', lines=True).sort_values('offer_type')

# Renaming columns
portfolio.rename(columns={'id': 'offer_id', 'difficulty': 'mms_unlock'}, inplace=True)
portfolio

,reward,channels,mms_unlock,duration,offer_type,offer_id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837


In [195]:
portfolio.to_csv('resources/portfolio.csv', index=False)

### Profile cleaning

In [196]:
profile = pd.read_json('resources/profile.json', orient='records', lines=True)

# Renaming column
profile.rename(columns={'id': 'customer_id'}, inplace=True)

# Convert column to datetime
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')
profile.head()

,gender,age,customer_id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN


In [197]:
# Check if every customer_id is unique
profile.shape, profile['customer_id'].nunique()

((17000, 5), 17000)

In [198]:
# Since age == 118 is a placeholder for null, we repalce it with nan
profile.loc[profile['age'] == 118, 'age'] = np.nan

In [199]:
profile.isnull().sum()

gender              2175
age                 2175
customer_id            0
became_member_on       0
income              2175
dtype: int64

In [200]:
# Percentage missing values
round(2175 / profile.shape[0] * 100, 1)

12.8

In [201]:
# Turns out, for some people, 'gender', 'age' and 'income' are all missing (around 12.8 %). We'll drop these columns
# after storing these names in a list
nan_customers = profile.loc[profile['gender'].isnull(), 'customer_id'].values.tolist()

In [202]:
profile.dropna(inplace=True)
profile.shape

(14825, 5)

In [203]:
profile.to_csv('resources/profile.csv', index=False)

### Transcript cleaning

In [204]:
transcript = pd.read_json('resources/transcript.json', orient='records', lines=True).sort_values(['person', 'time']).\
                                                                                     reset_index(drop=True)

# Rename columns
transcript.rename(columns={'person': 'customer_id', 'time': 'days_since_test_start'}, inplace=True)

# Convert hours to days
transcript['days_since_test_start'] = transcript['days_since_test_start'] / 24

transcript.head()

,customer_id,event,value,days_since_test_start
0,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},7.0
1,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},8.0
2,0009655768c64bdeb2e877511632db8f,transaction,{'amount': 22.16},9.5
3,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},14.0
4,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '3f207df678b143eea3cee63160fa8bed'},15.5


In [205]:
# Drop all nan_customers from transcript
transcript = transcript.loc[~transcript['customer_id'].isin(nan_customers)]

In [206]:
transcript.to_csv('resources/transcript.csv', index=False)